In [2]:
!pip install numpy pandas tensorflow music21 pydub librosa

In [3]:
import numpy as np
import pandas as pd
import os
from music21 import converter, instrument, note, chord, stream
from pydub import AudioSegment
import librosa
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense, Activation
from tensorflow.keras.utils import to_categorical

In [5]:
def convert_mp3_to_wav(mp3_path, wav_path):
    audio = AudioSegment.from_mp3(mp3_path)
    audio.export(wav_path, format="wav")

mp3_path = '/content/Kcee - Ojapiano (Official Video).mp3'
wav_path = '/content/temp_audio.wav'

convert_mp3_to_wav(mp3_path, wav_path)

In [6]:
def extract_features(wav_path):
    y, sr = librosa.load(wav_path)
    tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    rmse = librosa.feature.rms(y=y)
    spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
    spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    zcr = librosa.feature.zero_crossing_rate(y)
    mfcc = librosa.feature.mfcc(y=y, sr=sr)
    features = np.concatenate((chroma_stft, rmse, spec_cent, spec_bw, rolloff, zcr, mfcc))
    return features

features = extract_features(wav_path)

In [7]:
sequence_length = 100
n_vocab = features.shape[0]

network_input = []
network_output = []

for i in range(len(features[0]) - sequence_length):
    seq_in = features[:, i:i + sequence_length]
    seq_out = features[:, i + sequence_length]
    network_input.append(seq_in)
    network_output.append(seq_out)

n_patterns = len(network_input)

network_input = np.reshape(network_input, (n_patterns, sequence_length, n_vocab))
network_input = network_input / np.max(network_input)
network_output = np.reshape(network_output, (n_patterns, n_vocab))
network_output = network_output / np.max(network_output)

In [8]:
model = Sequential()
model.add(LSTM(512, input_shape=(network_input.shape[1], network_input.shape[2]), return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='mean_squared_error', optimizer='adam')

model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 100, 512)            │       1,126,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 100, 512)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 100, 512)            │       2,099,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 100, 512)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ (None, 512)                 │       2,099,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 37)                  │           9,509 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation (Activation)              │ (None, 37)                  │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,465,637 (20.85 MB)

 Trainable params: 5,465,637 (20.85 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
# Train the model
model.fit(network_input, network_output, epochs=10, batch_size=64)

# Save the model
model.save('music_generation_model.h5')

Epoch 1/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 909s 8s/step - loss: 0.0056
Epoch 2/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 893s 8s/step - loss: 0.0021
Epoch 3/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 922s 8s/step - loss: 0.0037
Epoch 4/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 894s 8s/step - loss: 0.0021
Epoch 5/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 908s 8s/step - loss: 0.0020
Epoch 6/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 924s 8s/step - loss: 0.0020
Epoch 7/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 920s 8s/step - loss: 0.0020
Epoch 8/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 890s 8s/step - loss: 0.0020
Epoch 9/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 907s 8s/step - loss: 0.0020
Epoch 10/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 935s 8s/step - loss: 0.0020


In [12]:
# Generate new music
def generate_notes(model, network_input, n_vocab, sequence_length):
    start = np.random.randint(0, len(network_input)-1)
    pattern = network_input[start]
    prediction_output = []

    for note_index in range(500):
        prediction_input = np.reshape(pattern, (1, len(pattern), n_vocab))
        prediction = model.predict(prediction_input, verbose=0)
        prediction_output.append(prediction[0])
        pattern = np.vstack((pattern[1:], prediction))

    return prediction_output

# Example usage
prediction_output = generate_notes(model, network_input, n_vocab, sequence_length)

In [13]:
# Convert prediction to MIDI file
def create_midi(prediction_output, output_file):
    offset = 0
    output_notes = []

    for pattern in prediction_output:
        new_note = note.Note()
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp=output_file)

# Example usage
create_midi(prediction_output, 'generated_music.mid')